# Technical Factor - Bollinger Bands (BOLL)

In [1]:
import dai
import pandas as pd

In [2]:
sd = '2025-01-01'
ed = '2026-01-01'

instrument_list = ['600519.SH']

In [3]:
sql = """
WITH
data_base AS (
    SELECT
        date,
        instrument,
        open,
        high,
        low,
        close,
        volume,
    FROM cn_stock_bar1d
),

data_factor AS (
    SELECT
        date,
        instrument,
        close,
        m_avg(close, 26) AS MIDDLE,
        MIDDLE + 2 * m_stddev(close, 26) AS UPPER,
        MIDDLE - 2 * m_stddev(close, 26) AS LOWER,
        (close - MIDDLE) / m_stddev(close, 26) AS Z,
        (UPPER - LOWER) / MIDDLE AS BANDWIDTH,
    FROM data_base
),

data_signal_trend AS (
    SELECT
        date,
        instrument,
        IF(close > MIDDLE AND m_lag(close, 1) <= m_lag(MIDDLE, 1), 1, 0) AS TRBY1,
        IF(close < MIDDLE AND m_lag(close, 1) >= m_lag(MIDDLE, 1), 1, 0) AS TRSL1,
        IF(MIDDLE > m_lag(MIDDLE, 1), 1, 0) AS TRBY2,
        IF(MIDDLE < m_lag(MIDDLE, 1), 1, 0) AS TRSL2,
    FROM data_factor
),

data_signal_momentum AS (
    SELECT
        date,
        instrument,
        IF(close > MIDDLE AND (close - MIDDLE) > (m_lag(close, 1) - m_lag(MIDDLE, 1)), 1, 0) AS MTBY1,
        IF(close < MIDDLE AND (close - MIDDLE) < (m_lag(close, 1) - m_lag(MIDDLE, 1)), 1, 0) AS MTSL1,
        IF(BANDWIDTH > m_lag(BANDWIDTH, 1), 1, 0) AS MTBY2,
        IF(BANDWIDTH < m_lag(BANDWIDTH, 1), 1, 0) AS MTSL2,
    FROM data_factor
),

data_signal_reversal AS (
    SELECT
        date,
        instrument,
        IF(close < LOWER AND close > m_lag(close, 1), 1, 0) AS RVBY1,
        IF(close > UPPER AND close < m_lag(close, 1), 1, 0) AS RVSL1,
        IF(Z < -2 AND (Z - m_lag(Z, 1)) > 0, 1, 0) AS RVBY2,
        IF(Z > 2 AND (Z - m_lag(Z, 1)) < 0, 1, 0) AS RVSL2,
    FROM data_factor
),

data_signal_breakout AS (
    SELECT
        date,
        instrument,
        IF(m_lag(close, 1) <= m_lag(UPPER, 1) AND close > UPPER, 1, 0) AS BKBY1,
        IF(m_lag(close, 1) >= m_lag(LOWER, 1) AND close < LOWER, 1, 0) AS BKSL1,
        IF(m_lag(BANDWIDTH, 1) <= m_lag(m_avg(BANDWIDTH, 26), 1) AND BANDWIDTH > m_avg(BANDWIDTH, 26), 1, 0) AS BKBY2,
        IF(m_lag(BANDWIDTH, 1) >= m_lag(m_avg(BANDWIDTH, 26), 1) AND BANDWIDTH < m_avg(BANDWIDTH, 26), 1, 0) AS BKSL2,
    FROM data_factor
),

data_combined AS (
    SELECT
        date,
        instrument,
        close,
        MIDDLE,
        UPPER,
        LOWER,
        Z,
        BANDWIDTH,
        TRBY1,
        TRSL1,
        TRBY2,
        TRSL2,
        MTBY1,
        MTSL1,
        MTBY2,
        MTSL2,
        RVBY1,
        RVSL1,
        RVBY2,
        RVSL2,
        BKBY1,
        BKSL1,
        BKBY2,
        BKSL2,
    FROM data_factor
    JOIN data_signal_trend    USING (date, instrument)
    JOIN data_signal_momentum USING (date, instrument)
    JOIN data_signal_reversal USING (date, instrument)
    JOIN data_signal_breakout USING (date, instrument)
)

SELECT *
FROM data_combined
QUALIFY COLUMNS(*) IS NOT NULL
ORDER BY date, instrument
"""

In [4]:
df = dai.query(sql, filters={"date":[sd,ed], "instrument":instrument_list}).df()
df

,date,instrument,close,MIDDLE,UPPER,LOWER,Z,BANDWIDTH,TRBY1,TRSL1,...,MTBY2,MTSL2,RVBY1,RVSL1,RVBY2,RVSL2,BKBY1,BKSL1,BKBY2,BKSL2
0,2025-02-14,600519.SH,12015.218095,11792.060875,12132.376319,11451.745431,1.311473,0.057719,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2025-02-17,600519.SH,11987.684917,11786.928951,12110.199414,11463.658489,1.242031,0.054852,0,0,...,0,1,0,0,0,0,0,0,0,0
2,2025-02-18,600519.SH,12015.218095,11786.928951,12110.199414,11463.658489,1.412372,0.054852,0,0,...,0,1,0,0,0,0,0,0,0,0
3,2025-02-19,600519.SH,12145.552664,11802.907468,12154.337296,11451.477640,1.950006,0.059550,0,0,...,1,0,0,0,0,0,0,0,0,0
4,2025-02-20,600519.SH,12007.072184,11813.497152,12172.517169,11454.477135,1.078352,0.060781,0,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213,2025-12-25,600519.SH,11945.482630,11906.432199,12182.654288,11630.210110,0.282747,0.046399,1,0,...,0,1,0,0,0,0,0,0,0,0
214,2025-12-26,600519.SH,11945.144750,11897.200484,12149.693637,11644.707332,0.379767,0.042446,0,0,...,0,1,0,0,0,0,0,0,0,0
215,2025-12-29,600519.SH,11842.682738,11884.190840,12109.232060,11659.149619,-0.368893,0.037872,0,1,...,0,1,0,0,0,0,0,0,0,0
216,2025-12-30,600519.SH,11738.953676,11871.855507,12091.988595,11651.722419,-1.207468,0.037085,0,0,...,0,1,0,0,0,0,0,0,0,0
